In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder \
.appName('Order Processing and Analytics Pipeline') \
.getOrCreate()

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

#PHASE 1 – Data Ingestion



1. Load the CSV file without schema inference.


In [3]:
orders_raw = spark.read \
.option("header", "true") \
.option("inferSchema", "false") \
.csv("orders.csv")

2. Print the schema.


In [4]:
orders_raw.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)



3. Count total records.


In [5]:
total_records = orders_raw.count()
print("Total records:", total_records)

Total records: 300000


4. Show sample rows.


In [6]:
orders_raw.show(5)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|
|ORD00000003|    C000003|  Bangalore|Electronics|     Laptop|   5558|2024-01-04|Completed|
|ORD00000004|    C000004|       Pune|       Home|AirPurifier|  33659|2024-01-05|Completed|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+
only showing top 5 rows


5. Explain why all columns must be treated as StringType initially.

In [7]:
#Load all columns as StringType initially when ingesting raw data (e.g., from CSVs).
#Avoid schema inference issues that can cause incorrect data types or job failures.
#Prevent data corruption by treating all values as strings during the first read.
#Ensure complete data ingestion without loss, even if formats are inconsistent.
#Enable controlled data processing: clean, validate, and cast columns explicitly later.
#Improves reliability for pipelines handling messy or unpredictable raw data.

#PHASE 2 – Data Cleaning

1. Remove leading and trailing spaces from:
city
category
product


In [8]:
orders_trimmed = orders_raw\
.withColumn("city_clean", trim(col("city")))\
.withColumn("category_clean", trim(col("category")))\
.withColumn("product_clean", trim(col("product")))
orders_trimmed.show(5)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled| hyderabad|       grocery|          Oil|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|      Pune|   Electronics|       Mobile|
|ORD00000003|    C000003|  Bangalore|Electronics|     Laptop|   5558|2024-01-04|Completed| Bangalore|   Electronics|       Laptop|
|ORD00000004|    C000004|       Pune|       Home|AirPurifier|  33659|2024-01-05|Com

2. Standardize text:
Convert city, category, and product to proper case.


In [9]:
orders_text_std = orders_trimmed\
.withColumn("city_clean", initcap(col("city_clean")))\
.withColumn("category_clean", initcap(col("category_clean")))\
.withColumn("product_clean", initcap(col("product_clean")))
orders_text_std.show(5)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled| Hyderabad|       Grocery|          Oil|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|      Pune|   Electronics|       Mobile|
|ORD00000003|    C000003|  Bangalore|Electronics|     Laptop|   5558|2024-01-04|Completed| Bangalore|   Electronics|       Laptop|
|ORD00000004|    C000004|       Pune|       Home|AirPurifier|  33659|2024-01-05|Com

3. Clean the amount column:
Remove commas.
Replace empty strings and invalid values with null.
Convert amount into IntegerType.
Rows with invalid amounts must not crash the pipeline.


In [10]:
orders_amount_clean = orders_text_std\
.withColumn("amount_clean", regexp_replace(col("amount"), ",", ""))\
.withColumn("amount_clean",
    when(col("amount_clean").rlike("^[0-9]+$"), col("amount_clean").cast(IntegerType()))\
    .otherwise(None)
)
orders_amount_clean.show(5)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled| Hyderabad|       Grocery|          Oil|        NULL|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|      Pune|   Electronics|       Mobile|       65358|
|ORD00000003|    C000003|  Bangalore|Electronics|     Laptop|   5558|2024-01-04|Completed| Bangalore|   Electronics|       Laptop|      

4. Clean the order_date column:

Support the following formats:
yyyy-MM-dd
dd/MM/yyyy
yyyy/MM/dd
Create a new column:

order_date_clean

with DateType.


In [15]:
orders_date_clean = orders_amount_clean\
.withColumn(
    "order_date_clean",
    coalesce(
    try_to_timestamp(col("order_date"), lit("yyyy-MM-dd")).cast(DateType()),
    try_to_timestamp(col("order_date"), lit("dd/MM/yyyy")).cast(DateType()),
    try_to_timestamp(col("order_date"), lit("yyyy/MM/dd")).cast(DateType())
))
orders_date_clean.show(5)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|order_date_clean|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+----------+--------------+-------------+------------+----------------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled| Hyderabad|       Grocery|          Oil|        NULL|      2024-01-01|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|      2024-01-02|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|      Pune|   Electronics|       Mobile|       65358|      2024-01-03|
|ORD00000003|    C000003|  Bangalo

5. The original columns must remain for auditing.

In [16]:
print("Schema of orders_date_clean (showing both original and cleaned columns):")
orders_date_clean.printSchema()

Schema of orders_date_clean (showing both original and cleaned columns):
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)
 |-- city_clean: string (nullable = true)
 |-- category_clean: string (nullable = true)
 |-- product_clean: string (nullable = true)
 |-- amount_clean: integer (nullable = true)
 |-- order_date_clean: date (nullable = true)



#PHASE 3 – Data Validation


1. Count how many records had invalid amounts.


In [18]:
invalid_amount_count = orders_date_clean.filter(col("amount_clean").isNull()).count()
print("Number of records with invalid amounts:", invalid_amount_count)

Number of records with invalid amounts: 25164


2. Count how many records had invalid dates.


In [19]:
invalid_date_count = orders_date_clean.filter(col("order_date_clean").isNull()).count()
print("Number of records with invalid dates: ", invalid_date_count)

Number of records with invalid dates:  2595


3. Identify duplicate order_id values.


In [20]:
duplicate_orders = orders_date_clean.groupBy("order_id").count().filter(col("count") > 1)
duplicate_orders.show()

+--------+-----+
|order_id|count|
+--------+-----+
+--------+-----+



4. Remove duplicates using order_id.


In [21]:
orders_deduped = orders_date_clean.dropDuplicates(["order_id"])
orders_deduped.show(5)

+-----------+-----------+---------+-----------+-------+------+----------+---------+----------+--------------+-------------+------------+----------------+
|   order_id|customer_id|     city|   category|product|amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|order_date_clean|
+-----------+-----------+---------+-----------+-------+------+----------+---------+----------+--------------+-------------+------------+----------------+
|ORD00000001|    C000001|     Pune|    Grocery|  Sugar| 35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|      2024-01-02|
|ORD00000007|    C000007|     Pune|    Grocery|   Rice| 45362|2024-01-08|Completed|      Pune|       Grocery|         Rice|       45362|      2024-01-08|
|ORD00000008|    C000008|Bangalore|    Fashion|  Jeans| 10563|2024-01-09|Completed| Bangalore|       Fashion|        Jeans|       10563|      2024-01-09|
|ORD00000010|    C000010|Bangalore|    Grocery|  Sugar| 66576|2024-01-11|Com

5. Filter only records with:

status = "Completed"



In [22]:
orders_completed = orders_deduped.filter(col("status") == "Completed")
orders_completed.show(5)

+-----------+-----------+---------+-----------+-------+------+----------+---------+----------+--------------+-------------+------------+----------------+
|   order_id|customer_id|     city|   category|product|amount|order_date|   status|city_clean|category_clean|product_clean|amount_clean|order_date_clean|
+-----------+-----------+---------+-----------+-------+------+----------+---------+----------+--------------+-------------+------------+----------------+
|ORD00000001|    C000001|     Pune|    Grocery|  Sugar| 35430|2024-01-02|Completed|      Pune|       Grocery|        Sugar|       35430|      2024-01-02|
|ORD00000007|    C000007|     Pune|    Grocery|   Rice| 45362|2024-01-08|Completed|      Pune|       Grocery|         Rice|       45362|      2024-01-08|
|ORD00000008|    C000008|Bangalore|    Fashion|  Jeans| 10563|2024-01-09|Completed| Bangalore|       Fashion|        Jeans|       10563|      2024-01-09|
|ORD00000010|    C000010|Bangalore|    Grocery|  Sugar| 66576|2024-01-11|Com

6. Record row counts at every stage.

In [23]:
print("initial", total_records)
print("after dedup", orders_deduped.count())
print("after filtering", orders_completed.count())

initial 300000
after dedup 300000
after filtering 285000


#PHASE 4 – Performance Engineering


1. Check the number of partitions.


In [24]:
orders_completed.rdd.getNumPartitions()

2

2. Run a groupBy on city and calculate total revenue.


In [25]:
city_revenue = orders_completed.groupBy("city_clean").agg(sum("amount_clean").alias("total_revenue"))
city_revenue.show(5)

+----------+-------------+
|city_clean|total_revenue|
+----------+-------------+
| Bangalore|   1628527093|
|   Chennai|   1629865247|
|    Mumbai|   1625518096|
|   Kolkata|   1624300497|
|      Pune|   1646196535|
+----------+-------------+
only showing top 5 rows


3. Use:

explain(True)

to analyze execution.


In [26]:
city_revenue.explain(True)

== Parsed Logical Plan ==
'Aggregate ['city_clean], ['city_clean, 'sum('amount_clean) AS total_revenue#1004]
+- Filter (status#24 = Completed)
   +- Deduplicate [order_id#17]
      +- Project [order_id#17, customer_id#18, city#19, category#20, product#21, amount#22, order_date#23, status#24, city_clean#116, category_clean#117, product_clean#118, amount_clean#162, coalesce(cast(try_to_timestamp(order_date#23, Some(yyyy-MM-dd), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#23, Some(dd/MM/yyyy), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#23, Some(yyyy/MM/dd), TimestampType, Some(Etc/UTC), false) as date)) AS order_date_clean#315]
         +- Project [order_id#17, customer_id#18, city#19, category#20, product#21, amount#22, order_date#23, status#24, city_clean#116, category_clean#117, product_clean#118, CASE WHEN RLIKE(amount_clean#161, ^[0-9]+$) THEN cast(amount_clean#161 as int) ELSE cast(null as int) END AS amount_cl

4. Identify where shuffle happens.


In [27]:
#By examining the "Physical Plan" section of the city_revenue.explain(True) output, we can identify the shuffle operations. Shuffles are indicated by Exchange operators.

#In this particular plan, shuffles happen at these stages:

#After the initial data read and projection: There's an Exchange hashpartitioning(order_id#17, 200) before the SortAggregate for deduplication. This shuffles the data by order_id to ensure all records for a given order_id are on the same partition for accurate deduplication.
#Before the final aggregation: There's another Exchange hashpartitioning(city_clean#1110, 200) before the final HashAggregate that calculates the total revenue per city. This shuffles the data by city_clean to bring all records belonging to the same city together on a single partition for efficient aggregation.


3

5. Repartition the dataset by city.


In [28]:
orders_repartitioned = orders_completed.repartition("city_clean")
orders_repartitioned.rdd.getNumPartitions()

3

6. Compare execution plans before and after repartition.
This phase exists to demonstrate understanding of Spark internals, not just outputs.

In [29]:
orders_completed.groupBy("city_clean").agg(sum("amount_clean").alias("total_revenue")).explain(True)
orders_repartitioned.groupBy("city_clean").agg(sum("amount_clean").alias("total_revenue")).explain(True)

== Parsed Logical Plan ==
'Aggregate ['city_clean], ['city_clean, 'sum('amount_clean) AS total_revenue#1485]
+- Filter (status#24 = Completed)
   +- Deduplicate [order_id#17]
      +- Project [order_id#17, customer_id#18, city#19, category#20, product#21, amount#22, order_date#23, status#24, city_clean#116, category_clean#117, product_clean#118, amount_clean#162, coalesce(cast(try_to_timestamp(order_date#23, Some(yyyy-MM-dd), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#23, Some(dd/MM/yyyy), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#23, Some(yyyy/MM/dd), TimestampType, Some(Etc/UTC), false) as date)) AS order_date_clean#315]
         +- Project [order_id#17, customer_id#18, city#19, category#20, product#21, amount#22, order_date#23, status#24, city_clean#116, category_clean#117, product_clean#118, CASE WHEN RLIKE(amount_clean#161, ^[0-9]+$) THEN cast(amount_clean#161 as int) ELSE cast(null as int) END AS amount_cl

#PHASE 5 – Analytics

1. Total revenue per city.


In [30]:
orders_completed.groupBy("city_clean").agg(sum("amount_clean").alias("total_revenue")).show(5)

+----------+-------------+
|city_clean|total_revenue|
+----------+-------------+
| Bangalore|   1628527093|
|   Chennai|   1629865247|
|    Mumbai|   1625518096|
|   Kolkata|   1624300497|
|      Pune|   1646196535|
+----------+-------------+
only showing top 5 rows


2. Total revenue per category.


In [31]:
orders_completed.groupBy("category_clean").agg(sum("amount_clean").alias("total_revenue")).show(5)

+--------------+-------------+
|category_clean|total_revenue|
+--------------+-------------+
|          Home|   2868467576|
|       Fashion|   2834182172|
|       Grocery|   2866272106|
|   Electronics|   2867568870|
+--------------+-------------+



3. Average order value per city.


In [32]:
orders_completed.groupBy("city_clean").agg(avg("amount_clean").alias("avg_order_value")).show(5)

+----------+------------------+
|city_clean|   avg_order_value|
+----------+------------------+
| Bangalore|44098.867908689645|
|   Chennai| 43628.27900315863|
|    Mumbai| 43723.75651612556|
|   Kolkata|43709.816662630175|
|      Pune|43930.204013556424|
+----------+------------------+
only showing top 5 rows


4. Top 10 products by revenue.


In [33]:
orders_completed.groupBy("product_clean").agg(sum("amount_clean").alias("total_revenue")).orderBy(col("total_revenue").desc()).limit(10).show()

+-------------+-------------+
|product_clean|total_revenue|
+-------------+-------------+
|          Oil|    963572869|
|       Laptop|    962496295|
|       Tablet|    960719999|
|       Vacuum|    959149427|
|        Mixer|    957140026|
|         Rice|    954494237|
|  Airpurifier|    952178123|
|        Jeans|    951286127|
|        Sugar|    948205000|
|        Shoes|    946799102|
+-------------+-------------+



5. Cities sorted by revenue descending.

In [34]:
city_revenue.orderBy(col("total_revenue").desc()).show(5)

+----------+-------------+
|city_clean|total_revenue|
+----------+-------------+
|      Pune|   1646196535|
| Hyderabad|   1642443340|
|     Delhi|   1639639916|
|   Chennai|   1629865247|
| Bangalore|   1628527093|
+----------+-------------+
only showing top 5 rows


#PHASE 6 – Window Functions


1. Rank cities by revenue.


In [40]:
city_rank_window = Window.orderBy(col("total_revenue").desc())
city_ranked = city_revenue.withColumn("rank", row_number().over(city_rank_window))
city_ranked.show(5)

+----------+-------------+----+
|city_clean|total_revenue|rank|
+----------+-------------+----+
|      Pune|   1646196535|   1|
| Hyderabad|   1642443340|   2|
|     Delhi|   1639639916|   3|
|   Chennai|   1629865247|   4|
| Bangalore|   1628527093|   5|
+----------+-------------+----+
only showing top 5 rows


2. Rank products inside each category by revenue.


In [37]:
category_product_rank_window = Window.partitionBy("category_clean").orderBy(col("total_revenue").desc())

3. Find the top product for every category.


In [38]:
product_revenue = orders_completed.groupBy("category_clean", "product_clean").agg(sum("amount_clean").alias("total_revenue"))
top_products = product_revenue.withColumn("rank", row_number().over(category_product_rank_window)).filter(col("rank") == 1).drop("rank")
top_products.show(5)

+--------------+-------------+-------------+
|category_clean|product_clean|total_revenue|
+--------------+-------------+-------------+
|   Electronics|       Laptop|    962496295|
|       Fashion|        Jeans|    951286127|
|       Grocery|          Oil|    963572869|
|          Home|       Vacuum|    959149427|
+--------------+-------------+-------------+



4. Identify the top 3 performing cities.

In [41]:
city_ranked.filter(col("rank") <= 3).show()

+----------+-------------+----+
|city_clean|total_revenue|rank|
+----------+-------------+----+
|      Pune|   1646196535|   1|
| Hyderabad|   1642443340|   2|
|     Delhi|   1639639916|   3|
+----------+-------------+----+



#PHASE 7 – Broadcast Join



In [42]:
city_region = spark.createDataFrame([
    ("Delhi", "North"),
    ("Mumbai", "West"),
    ("Bangalore", "South"),
    ("Hyderabad", "South"),
    ("Pune", "West"),
    ("Chennai", "South"),
    ("Kolkata", "East")
], ["city_clean", "region"])

1. Join the orders data with this city-region dataset.


In [43]:
from pyspark.sql.functions import broadcast
orders_with_regions = orders_completed.join(broadcast(city_region), "city_clean")
orders_with_regions.show(5)

+----------+-----------+-----------+---------+-----------+-------+------+----------+---------+--------------+-------------+------------+----------------+------+
|city_clean|   order_id|customer_id|     city|   category|product|amount|order_date|   status|category_clean|product_clean|amount_clean|order_date_clean|region|
+----------+-----------+-----------+---------+-----------+-------+------+----------+---------+--------------+-------------+------------+----------------+------+
|      Pune|ORD00000001|    C000001|     Pune|    Grocery|  Sugar| 35430|2024-01-02|Completed|       Grocery|        Sugar|       35430|      2024-01-02|  West|
|      Pune|ORD00000007|    C000007|     Pune|    Grocery|   Rice| 45362|2024-01-08|Completed|       Grocery|         Rice|       45362|      2024-01-08|  West|
| Bangalore|ORD00000008|    C000008|Bangalore|    Fashion|  Jeans| 10563|2024-01-09|Completed|       Fashion|        Jeans|       10563|      2024-01-09| South|
| Bangalore|ORD00000010|    C00001

2. Apply broadcast join explicitly.


In [44]:
from pyspark.sql.functions import broadcast
orders_with_regions = orders_completed.join(broadcast(city_region), "city_clean")
orders_with_regions.show(5)

+----------+-----------+-----------+---------+-----------+-------+------+----------+---------+--------------+-------------+------------+----------------+------+
|city_clean|   order_id|customer_id|     city|   category|product|amount|order_date|   status|category_clean|product_clean|amount_clean|order_date_clean|region|
+----------+-----------+-----------+---------+-----------+-------+------+----------+---------+--------------+-------------+------------+----------------+------+
|      Pune|ORD00000001|    C000001|     Pune|    Grocery|  Sugar| 35430|2024-01-02|Completed|       Grocery|        Sugar|       35430|      2024-01-02|  West|
|      Pune|ORD00000007|    C000007|     Pune|    Grocery|   Rice| 45362|2024-01-08|Completed|       Grocery|         Rice|       45362|      2024-01-08|  West|
| Bangalore|ORD00000008|    C000008|Bangalore|    Fashion|  Jeans| 10563|2024-01-09|Completed|       Fashion|        Jeans|       10563|      2024-01-09| South|
| Bangalore|ORD00000010|    C00001

3. Verify using the physical plan that:

BroadcastHashJoin

is used.


In [45]:
orders_with_regions.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city_clean])
:- Filter (status#24 = Completed)
:  +- Deduplicate [order_id#17]
:     +- Project [order_id#17, customer_id#18, city#19, category#20, product#21, amount#22, order_date#23, status#24, city_clean#116, category_clean#117, product_clean#118, amount_clean#162, coalesce(cast(try_to_timestamp(order_date#23, Some(yyyy-MM-dd), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#23, Some(dd/MM/yyyy), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#23, Some(yyyy/MM/dd), TimestampType, Some(Etc/UTC), false) as date)) AS order_date_clean#315]
:        +- Project [order_id#17, customer_id#18, city#19, category#20, product#21, amount#22, order_date#23, status#24, city_clean#116, category_clean#117, product_clean#118, CASE WHEN RLIKE(amount_clean#161, ^[0-9]+$) THEN cast(amount_clean#161 as int) ELSE cast(null as int) END AS amount_clean#162]
:           +- Project [order_id#17, 

4. Explain why broadcast join is efficient in this case.

In [ ]:
#small lookup table
#avoids shuffle
#sent to all executors
#faster joins at scale

#PHASE 8 – UDF



Create a classification based on amount:

amount >= 80000 → High
amount >= 40000 → Medium
else → Low

Add a new column:

order_value_category

Analyze distribution.

In [47]:
orders_classified = orders_with_regions\
.withColumn(
    "order_value_category",
    when(col("amount_clean") >= 80000, "High")\
    .when(col("amount_clean") >= 40000, "Medium")\
    .otherwise("Low")
)
orders_classified.show(5)

+----------+-----------+-----------+---------+-----------+-------+------+----------+---------+--------------+-------------+------------+----------------+------+--------------------+
|city_clean|   order_id|customer_id|     city|   category|product|amount|order_date|   status|category_clean|product_clean|amount_clean|order_date_clean|region|order_value_category|
+----------+-----------+-----------+---------+-----------+-------+------+----------+---------+--------------+-------------+------------+----------------+------+--------------------+
|      Pune|ORD00000001|    C000001|     Pune|    Grocery|  Sugar| 35430|2024-01-02|Completed|       Grocery|        Sugar|       35430|      2024-01-02|  West|                 Low|
|      Pune|ORD00000007|    C000007|     Pune|    Grocery|   Rice| 45362|2024-01-08|Completed|       Grocery|         Rice|       45362|      2024-01-08|  West|              Medium|
| Bangalore|ORD00000008|    C000008|Bangalore|    Fashion|  Jeans| 10563|2024-01-09|Comple

In [48]:
orders_classified.groupBy("order_value_category").count().show()

+--------------------+------+
|order_value_category| count|
+--------------------+------+
|                High| 27936|
|                 Low|145699|
|              Medium|111365|
+--------------------+------+



#PHASE 9 – RDD


1. Convert the cleaned DataFrame to RDD.


In [49]:
orders_rdd = orders_completed.select("city_clean", "amount_clean").rdd

2. Compute:
Total revenue using reduce.
Orders per city using map and reduce.


In [52]:
total_revenue = orders_rdd.map(lambda x: x[1] if x[1] is not None else 0).reduce(lambda x, y: x + y)
print("Total revenue:", total_revenue)

Total revenue: 11436490724


3. Explain why DataFrames are preferred over RDDs for analytics.

In [ ]:
#DataFrames are faster and optimized using Catalyst and Tungsten.
#Support SQL queries and rich built-in functions.
#Schema enforcement ensures structured data and better performance.
#Columnar storage improves memory and speed.

#PHASE 10 – Caching


1. Identify datasets reused in multiple queries.


In [53]:
# Datasets heavily reused in multiple queries:
# 1. orders_completed: Used for all analytics (Phase 5), window functions (Phase 6), broadcast joins (Phase 7), UDF (Phase 8), and RDD conversions (Phase 9).
# 2. city_revenue: Derived from orders_completed and then used for ranking cities (Phase 6) and sorting (Phase 5).

print("Datasets identified for potential caching:")
print("- orders_completed (used extensively for analytics, joins, and RDD operations)")
print("- city_revenue (used for further analysis and window functions)")

Datasets identified for potential caching:
- orders_completed (used extensively for analytics, joins, and RDD operations)
- city_revenue (used for further analysis and window functions)


2. Apply cache().


In [54]:
orders_completed.cache()
city_revenue.cache()

print("orders_completed DataFrame is cached:", orders_completed.is_cached)
print("city_revenue DataFrame is cached:", city_revenue.is_cached)

orders_completed DataFrame is cached: True
city_revenue DataFrame is cached: True


3. Execute multiple aggregations.


In [55]:
print("Executing aggregations on cached DataFrames...")

# Aggregation 1: Total revenue per category (on cached orders_completed)
orders_completed.groupBy("category_clean").agg(sum("amount_clean").alias("total_revenue")).show(5)

# Aggregation 2: Average order value per city (on cached orders_completed)
orders_completed.groupBy("city_clean").agg(avg("amount_clean").alias("avg_order_value")).show(5)

# Aggregation 3: Max revenue per city (on cached city_revenue, which itself is cached)
city_revenue.agg(max("total_revenue")).show()

print("Aggregations executed.")

Executing aggregations on cached DataFrames...
+--------------+-------------+
|category_clean|total_revenue|
+--------------+-------------+
|          Home|   2868467576|
|       Fashion|   2834182172|
|       Grocery|   2866272106|
|   Electronics|   2867568870|
+--------------+-------------+

+----------+------------------+
|city_clean|   avg_order_value|
+----------+------------------+
| Bangalore|44098.867908689645|
|   Chennai| 43628.27900315863|
|    Mumbai| 43723.75651612556|
|   Kolkata|43709.816662630175|
|      Pune|43930.204013556424|
+----------+------------------+
only showing top 5 rows
+------------------+
|max(total_revenue)|
+------------------+
|        1646196535|
+------------------+

Aggregations executed.


4. Compare performance.


In [56]:
import time

print("--- Comparing Performance of Cached vs. Uncached DataFrames ---")

# Unpersist DataFrames first to ensure a clean comparison
orders_completed.unpersist()
city_revenue.unpersist()
print("\nDataFrames unpersisted for baseline comparison.")

# --- Baseline: Execute aggregations without caching ---
print("\nExecuting aggregations WITHOUT caching...")
start_time_uncached = time.time()

orders_completed.groupBy("category_clean").agg(sum("amount_clean").alias("total_revenue")).collect()
orders_completed.groupBy("city_clean").agg(avg("amount_clean").alias("avg_order_value")).collect()
city_revenue.agg(max("total_revenue")).collect()

end_time_uncached = time.time()
duration_uncached = end_time_uncached - start_time_uncached
print(f"Aggregations WITHOUT caching took: {duration_uncached:.4f} seconds")

# --- Re-cache DataFrames ---
orders_completed.cache()
city_revenue.cache()
# Trigger caching with an action
orders_completed.count()
city_revenue.count()
print("\nDataFrames re-cached.")

# --- Cached: Execute aggregations WITH caching ---
print("\nExecuting aggregations WITH caching...")
start_time_cached = time.time()

orders_completed.groupBy("category_clean").agg(sum("amount_clean").alias("total_revenue")).collect()
orders_completed.groupBy("city_clean").agg(avg("amount_clean").alias("avg_order_value")).collect()
city_revenue.agg(max("total_revenue")).collect()

end_time_cached = time.time()
duration_cached = end_time_cached - start_time_cached
print(f"Aggregations WITH caching took: {duration_cached:.4f} seconds")

print("\nPerformance Comparison:")
print(f"Uncached duration: {duration_uncached:.4f} seconds")
print(f"Cached duration:   {duration_cached:.4f} seconds")
if duration_uncached > 0:
    speedup = duration_uncached / duration_cached
    print(f"Caching provided a speedup of approximately {speedup:.2f}x")


--- Comparing Performance of Cached vs. Uncached DataFrames ---

DataFrames unpersisted for baseline comparison.

Executing aggregations WITHOUT caching...
Aggregations WITHOUT caching took: 8.5717 seconds

DataFrames re-cached.

Executing aggregations WITH caching...
Aggregations WITH caching took: 7.1447 seconds

Performance Comparison:
Uncached duration: 8.5717 seconds
Cached duration:   7.1447 seconds
Caching provided a speedup of approximately 1.20x


5. Unpersist after use.
Explain why unnecessary caching is dangerous.

In [57]:
orders_completed.unpersist()
#memory pressure
#executor oom
#cache eviction
#slower jobs

DataFrame[order_id: string, customer_id: string, city: string, category: string, product: string, amount: string, order_date: string, status: string, city_clean: string, category_clean: string, product_clean: string, amount_clean: int, order_date_clean: date]

#PHASE 11 – Storage Formats


1. Write cleaned dataset to:

Parquet

Partitioned by:

city



In [58]:
orders_completed.write.mode("overwrite").partitionBy("city_clean").parquet("orders_parquet")

2. Write aggregated datasets to:

ORC



In [60]:
city_revenue.write.mode("overwrite").orc("city_revenue_orc")

3. Read both formats back and validate:
Schema
Row counts


In [62]:
spark.read.parquet("orders_parquet").printSchema()
spark.read.parquet("orders_parquet").count()
spark.read.orc("city_revenue_orc").printSchema()
spark.read.orc("city_revenue_orc").count()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)
 |-- category_clean: string (nullable = true)
 |-- product_clean: string (nullable = true)
 |-- amount_clean: integer (nullable = true)
 |-- order_date_clean: date (nullable = true)
 |-- city_clean: string (nullable = true)

root
 |-- city_clean: string (nullable = true)
 |-- total_revenue: long (nullable = true)



7

4. Compare size and performance against CSV.

In [64]:
import os
import time
import builtins # Import builtins module to access Python's built-in functions explicitly

# --- File Size Comparison ---
print("\n--- File Size Comparison ---")

csv_file_size = os.path.getsize("orders.csv")
parquet_dir_size = builtins.sum(f.stat().st_size for f in os.scandir("orders_parquet") if f.is_file())
orc_dir_size = builtins.sum(f.stat().st_size for f in os.scandir("city_revenue_orc") if f.is_file())

print(f"CSV (orders.csv) size: {csv_file_size / (1024*1024):.2f} MB")
print(f"Parquet (orders_parquet) directory size: {parquet_dir_size / (1024*1024):.2f} MB")
print(f"ORC (city_revenue_orc) directory size: {orc_dir_size / (1024*1024):.2f} MB")

# --- Read Performance Comparison ---
print("\n--- Read Performance Comparison ---")

# CSV Read Time
start_time = time.time()
spark.read.option("header", "true").option("inferSchema", "false").csv("orders.csv").count()
end_time = time.time()
print(f"CSV read time: {end_time - start_time:.4f} seconds")

# Parquet Read Time
start_time = time.time()
spark.read.parquet("orders_parquet").count()
end_time = time.time()
print(f"Parquet read time: {end_time - start_time:.4f} seconds")

# ORC Read Time
start_time = time.time()
spark.read.orc("city_revenue_orc").count()
end_time = time.time()
print(f"ORC read time: {end_time - start_time:.4f} seconds")


--- File Size Comparison ---
CSV (orders.csv) size: 19.98 MB
Parquet (orders_parquet) directory size: 0.00 MB
ORC (city_revenue_orc) directory size: 0.00 MB

--- Read Performance Comparison ---
CSV read time: 0.7060 seconds
Parquet read time: 0.3689 seconds
ORC read time: 0.1949 seconds


#PHASE 12 – Debugging


Explain why this breaks:

df = df.filter(df.amount > 50000).show()

And why after this line df is no longer a DataFrame.

In [ ]:
#The explanation for why df = df.filter(df.amount > 50000).show() breaks, and why df is no longer a DataFrame afterward, can be found in cell OU_cAY0uangU. It details the distinction between Spark transformations and actions, and how the show() action returns None, leading to reassignment of df to NoneType.

#PHASE 13 – Final Validation


1. Confirm:

amount is IntegerType

order_date_clean is DateType

No nulls in critical business fields.


In [65]:
print("--- Final Data Validation ---")

# Confirm amount_clean is IntegerType and order_date_clean is DateType
print("\nSchema verification for critical columns:")
orders_completed.select("amount_clean", "order_date_clean").printSchema()

# Check for nulls in critical business fields (amount_clean, order_date_clean)
print("\nChecking for nulls in critical business fields (amount_clean, order_date_clean):")

# For amount_clean
null_amount_count = orders_completed.filter(col("amount_clean").isNull()).count()
print(f"Number of records with null amount_clean: {null_amount_count}")

# For order_date_clean
null_order_date_count = orders_completed.filter(col("order_date_clean").isNull()).count()
print(f"Number of records with null order_date_clean: {null_order_date_count}")

if null_amount_count == 0 and null_order_date_count == 0:
    print("\nValidation successful: No nulls found in amount_clean or order_date_clean.")
else:
    print("\nValidation Alert: Nulls found in critical business fields.")

--- Final Data Validation ---

Schema verification for critical columns:
root
 |-- amount_clean: integer (nullable = true)
 |-- order_date_clean: date (nullable = true)


Checking for nulls in critical business fields (amount_clean, order_date_clean):
Number of records with null amount_clean: 23905
Number of records with null order_date_clean: 2465

Validation Alert: Nulls found in critical business fields.


2. Document:

Cleaning strategy

Performance strategy

Debugging learnings

In [ ]:
#Strategies and Learnings
#1. Cleaning Strategy

#StringType Ingestion: Load all columns as StringType to avoid schema inference issues and ensure complete data capture.
#Text Standardization: Trim spaces and apply proper case to city, category, and product columns.
#Amount Cleaning: Remove commas, validate numeric strings, cast to IntegerType; invalid values → NULL.
#Date Handling: Use coalesce with try_to_timestamp for multiple formats; unparseable dates → NULL.
#Preserve Raw Data: Create _clean columns for traceability.
#Deduplication: Remove duplicates based on order_id.
#Filter Valid Status: Keep only status = 'Completed'.

#2. Performance Strategy

#Partitioning: Monitor and repartition by common keys (e.g., city) to reduce shuffle.
#Broadcast Join: Use broadcast() for small lookup tables to avoid shuffling large DataFrames.
#Caching: Cache reused DataFrames for faster queries; demonstrated ~1.2x speedup.
#Unpersist: Free memory after use to prevent OOM issues.

#3. Debugging Learnings

#when().otherwise(): Ensure correct syntax and nesting.
#Date Parsing: Use try_to_timestamp instead of strict to_date.
#lit() for Constants: Wrap string literals to avoid AnalysisException.
#SQL Execution: Use spark.sql("query"), not spark.sql.session.sql().
#show() Behavior: show() is an action, returns None—don’t reassign DataFrame.
#sum() Conflict: Differentiate Python’s sum() vs PySpark’s sum().
#Handle None in RDDs: Replace None with defaults before aggregation.